In [ ]:
import pandas as pd
from collections import deque

# ============================================================
# 1. PARAMETERS
# ============================================================
WEEKS = list(range(1, 16))
PLAYERS = ["Retailer", "Wholesaler", "Distributor", "Factory"]

DEMAND = {
    1: 15, 2: 15, 3: 15, 4: 15,
    5: 25, 6: 25, 7: 25, 8: 80,
    9: 30, 10: 18, 11: 12, 12: 22,
    13: 35, 14: 18, 15: 16
}

INITIAL_INVENTORY = 15

# Base-stock target levels
TARGET_INVENTORY = {
    "Retailer": 40,
    "Wholesaler": 45,
    "Distributor": 50,
    "Factory": 55
}

SCENARIOS = {
    "baseline": {"shipping_delay": 2, "info_share": False},
    "info_share": {"shipping_delay": 2, "info_share": True},
    "lean": {"shipping_delay": 0, "info_share": False},
    "lean_info_share": {"shipping_delay": 0, "info_share": True},
}

# ============================================================
# 2. BEER GAME ENVIRONMENT
# ============================================================
class BeerGameRobotEnv:
    def __init__(self, shipping_delay):
        self.inventory = {p: INITIAL_INVENTORY for p in PLAYERS}
        self.backlog = {p: 0 for p in PLAYERS}

        self.orders = {p: [] for p in PLAYERS}
        self.inventory_hist = {p: [] for p in PLAYERS}
        self.backlog_hist = {p: [] for p in PLAYERS}

        self.pipeline = (
            {p: deque([INITIAL_INVENTORY] * shipping_delay) for p in PLAYERS}
            if shipping_delay > 0 else None
        )

    def receive_shipments(self):
        if self.pipeline:
            for p in PLAYERS:
                self.inventory[p] += self.pipeline[p].popleft()

    def fulfill_demand(self, observed):
        for p in PLAYERS:
            total_demand = observed[p] + self.backlog[p]
            shipped = min(self.inventory[p], total_demand)
            self.inventory[p] -= shipped
            self.backlog[p] = total_demand - shipped

            self.inventory_hist[p].append(self.inventory[p])
            self.backlog_hist[p].append(self.backlog[p])

    def place_orders(self, orders):
        for p in PLAYERS:
            qty = max(0, int(orders[p]))
            self.orders[p].append(qty)
            if self.pipeline:
                self.pipeline[p].append(qty)

# ============================================================
# 3. ROBOT DECISION LOGIC
# ============================================================
def robot_decision(player, inventory, backlog, incoming):
    inventory_position = inventory + incoming - backlog
    order = TARGET_INVENTORY[player] - inventory_position
    return max(0, order)

# ============================================================
# 4. RUN ONE SCENARIO
# ============================================================
def run_robot_scenario(scenario_name):
    config = SCENARIOS[scenario_name]
    env = BeerGameRobotEnv(config["shipping_delay"])

    last_orders = {p: INITIAL_INVENTORY for p in PLAYERS}

    for week in WEEKS:
        env.receive_shipments()

        observed = {}
        for i, p in enumerate(PLAYERS):
            if p == "Retailer":
                observed[p] = DEMAND[week]
            else:
                observed[p] = last_orders[PLAYERS[i - 1]]

        env.fulfill_demand(observed)

        decisions = {}
        for p in PLAYERS:
            incoming = env.pipeline[p][0] if env.pipeline else 0
            decisions[p] = robot_decision(
                p,
                env.inventory[p],
                env.backlog[p],
                incoming
            )

        env.place_orders(decisions)
        last_orders = decisions.copy()

    return env

# ============================================================
# 5. RUN ALL SCENARIOS + SAVE TO EXCEL
# ============================================================
for scenario in SCENARIOS:
    env = run_robot_scenario(scenario)

    rows = []
    for p in PLAYERS:
        for w in range(len(env.orders[p])):
            rows.append({
                "Scenario": scenario,
                "Week": w + 1,
                "Player": p,
                "Order": env.orders[p][w],
                "Inventory": env.inventory_hist[p][w],
                "Backlog": env.backlog_hist[p][w]
            })

    df = pd.DataFrame(rows)
    df.to_excel(f"ROBOT_Beer_Game_{scenario}.xlsx", index=False)

print("Robot-based simulations completed.")
